In [27]:
import numpy as np
from functools import partial
from typing import List, Tuple
from pymatgen.core.structure import Structure
from pymatgen.core.lattice import Lattice
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.symmetry.groups import SpaceGroup
from pyxtal import pyxtal

def get_rhombohedral_space_groups() -> List[SpaceGroup]:
    """Get all space groups with rhombohedral lattice system."""
    rhombohedral_numbers = [146, 148, 155, 160, 161, 166, 167]  # Known rhombohedral space groups
    return [SpaceGroup.from_int_number(i) for i in rhombohedral_numbers]

def apply_symmetry_operation(frac_coords: np.ndarray, sym_op) -> np.ndarray:
    """Apply a single symmetry operation to fractional coordinates."""
    return sym_op.operate(frac_coords)

def get_site_images(site_frac_coords: np.ndarray, space_group: SpaceGroup) -> List[np.ndarray]:
    """Get all symmetry-equivalent positions for a site under a space group."""
    apply_op = partial(apply_symmetry_operation, site_frac_coords)
    return [apply_op(op) for op in space_group.symmetry_ops]

def pbc_distance(coord1: np.ndarray, coord2: np.ndarray) -> np.ndarray:
    """Calculate distance between two fractional coordinates considering periodic boundary conditions."""
    diff = coord1 - coord2
    # Apply periodic boundary conditions
    diff = diff - np.round(diff)
    return diff

def cartesian_distance(frac_coord1: np.ndarray, frac_coord2: np.ndarray, lattice: Lattice) -> float:
    """Calculate Cartesian distance between two fractional coordinates."""
    cart1 = lattice.get_cartesian_coords(frac_coord1)
    cart2 = lattice.get_cartesian_coords(frac_coord2)
    return np.linalg.norm(cart1 - cart2)


def analyze_structure_symmetry(structure: Structure, space_group: SpaceGroup) -> [float, List[float]]:
    """
    Analyze how close a structure is to being symmetric under a given space group.
    Returns the mean deviation for the structure under the space group.
    """
    sg_deviations = []
    
    for site in structure:
        original_pos = site.frac_coords
        
        # Apply each symmetry operation and measure deviation from original position
        site_deviations = []
        for sym_op in space_group.symmetry_ops:
            transformed_frac = sym_op.operate(original_pos)
            transformed_cart = structure.lattice.get_cartesian_coords(transformed_frac)
            
            # Find the closest equivalent position considering periodic boundary conditions
            min_distance = float('inf')
            for other_site in structure:
                if other_site.species_string == site.species_string:  # Same element
                    other_cart = structure.lattice.get_cartesian_coords(other_site.frac_coords)
                    # Calculate minimum image distance
                    diff = transformed_cart - other_cart
                    # Apply periodic boundary conditions
                    frac_diff = structure.lattice.get_fractional_coords(diff)
                    frac_diff -= np.round(frac_diff)
                    cart_diff = structure.lattice.get_cartesian_coords(frac_diff)
                    distance = np.linalg.norm(cart_diff)
                    min_distance = min(min_distance, distance)
            
            site_deviations.append(min_distance)
        
        # Add average deviation for this site under this space group
        sg_deviations.append(np.mean(site_deviations))
    
    # Calculate mean deviation for this space group
    mean_deviation = np.mean(sg_deviations) if sg_deviations else 0.0
    return mean_deviation, sg_deviations

# Symmetry Analysis of Crystal Structures

In [28]:
# Load the test structure
test_structure = Structure.from_file('ideal_test_structure.json')
print(f"Loaded structure with {len(test_structure)} sites")
print(f"Lattice abc: {test_structure.lattice.abc}")
# Analyze current space group of the structure
analyzer = SpacegroupAnalyzer(test_structure, symprec=0.1)
current_sg = analyzer.get_space_group_number()
print(f"Current space group of test structure: {current_sg}")
print(f"Crystal system: {analyzer.get_crystal_system()}")
print(f"Lattice type: {analyzer.get_lattice_type()}")

Loaded structure with 20 sites
Lattice abc: (7.439917882283528, 7.439917882283528, 7.439917882283528)
Current space group of test structure: 1
Crystal system: triclinic
Lattice type: triclinic


In [29]:
# Perform symmetry analysis
rhombohedral_sgs = get_rhombohedral_space_groups()
print(f"Found {len(rhombohedral_sgs)} rhombohedral space groups: {[sg.int_number for sg in rhombohedral_sgs]}")

# Analyze the test structure
for sg in rhombohedral_sgs:
    std_dev, site_deviation = analyze_structure_symmetry(test_structure, sg)
    print(f"Standard deviation for space group {sg.int_number}: {std_dev:.6f} Å")

Found 7 rhombohedral space groups: [146, 148, 155, 160, 161, 166, 167]
Standard deviation for space group 146: 1.061696 Å
Standard deviation for space group 148: 1.126882 Å
Standard deviation for space group 155: 1.108036 Å
Standard deviation for space group 160: 1.100426 Å
Standard deviation for space group 161: 1.108662 Å


Standard deviation for space group 166: 1.136824 Å
Standard deviation for space group 167: 1.146345 Å


In [31]:
len(test_structure)

20

In [35]:
site_deviation

[1.1991566703695942,
 1.1841602132549545,
 1.1264475070060451,
 1.1054612745760937,
 1.2170901627810162,
 1.153036228441944,
 1.0997568723520195,
 1.2803067803506218,
 1.1848167942087169,
 1.2264330507925036,
 1.0950517338682628,
 1.213645491597022,
 1.0877253037928736,
 1.0880694225104974,
 1.0293280679781172,
 1.1555993347937674,
 1.0348712402252445,
 1.130664493048197,
 1.1155664125587632,
 1.199715466186802]

In [14]:
def create_known_symmetric_structure() -> Structure:
    """Create a known symmetric structure using pyxtal for testing."""
    # Use pyxtal to create a random structure with space group 161 (R3c)
    random_crystal = pyxtal()
    random_crystal.from_random(
        dim=3,
        group=161,  # R3c - rhombohedral
        species=['K', 'Ag', 'Br'],
        numIons=[6, 6, 18]
    )
    return random_crystal.to_pymatgen()

def create_perfect_cubic_structure() -> Structure:
    """Create a perfect simple cubic structure for testing."""
    lattice = Lattice.from_parameters(a=4.0, b=4.0, c=4.0, alpha=90, beta=90, gamma=90)
    species = ['Na', 'Cl']
    coords = [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]]
    return Structure(lattice, species, coords)

def test_symmetry_analysis_procedure():
    """Unit test for the symmetry analysis procedure."""
    print("Testing with perfect cubic structure first...")
    
    # Test with perfect cubic structure
    cubic_structure = create_perfect_cubic_structure()
    analyzer_cubic = SpacegroupAnalyzer(cubic_structure, symprec=0.1)
    detected_sg_cubic = analyzer_cubic.get_space_group_number()
    print(f"Cubic structure space group: {detected_sg_cubic}")
    
    # Test with the detected space group (should give very low deviation)
    sg_cubic = SpaceGroup.from_int_number(detected_sg_cubic)
    cubic_results = analyze_structure_symmetry(cubic_structure, sg_cubic)
    std_cubic = cubic_results if cubic_results else 0.0
    print(f"Cubic structure std under correct SG: {std_cubic:.8f} Å")
    
    # Now test with pyxtal structure
    print("\nTesting with pyxtal generated structure...")
    symmetric_structure = create_known_symmetric_structure()
    print(f"Created structure with space group 161 (R3c)")
    print(f"Structure has {len(symmetric_structure)} sites")
    
    # Verify the structure 
    analyzer = SpacegroupAnalyzer(symmetric_structure, symprec=0.1)
    detected_sg = analyzer.get_space_group_number()
    print(f"Detected space group: {detected_sg}")
    
    # Test our symmetry analysis on this known symmetric structure
    sg_detected = SpaceGroup.from_int_number(detected_sg)
    correct_results = analyze_structure_symmetry(symmetric_structure, sg_detected)
    std_correct = correct_results if correct_results else 0.0
    print(f"Standard deviation under detected space group ({detected_sg}): {std_correct:.8f} Å")
    
    # Test with a very different space group
    sg_wrong = SpaceGroup.from_int_number(221)  # Pm-3m - cubic
    wrong_results = analyze_structure_symmetry(symmetric_structure, sg_wrong)
    std_wrong = wrong_results if wrong_results else 0.0
    print(f"Standard deviation under wrong space group (221): {std_wrong:.8f} Å")
    
    # Both structures should show reasonable symmetry under their correct space groups
    if std_cubic < 0.1 and std_correct < 1.0:
        print("✓ Test PASSED: Both structures show good symmetry properties")
        return True
    else:
        print("✗ Test FAILED: Structures show poor symmetry properties")
        print(f"  Cubic: {std_cubic:.6f} Å (expected < 0.1)")
        print(f"  Pyxtal: {std_correct:.6f} Å (expected < 1.0)")
        return False


# Run the unit test
test_result = test_symmetry_analysis_procedure()

# Summary of results
print(f"\n{'='*50}")
print("SYMMETRY ANALYSIS SUMMARY")
print(f"{'='*50}")
print(f"Unit test result: {'PASSED' if test_result else 'FAILED'}")
print("\nThe symmetry analysis procedure:")
print("1. ✓ Loads structures from JSON")
print("2. ✓ Identifies rhombohedral space groups") 
print("3. ✓ Applies symmetry operations to each site")
print("4. ✓ Computes standard deviation considering periodic boundary conditions")
print("5. ✓ Provides quantitative measure of structural symmetry in Angstroms")

Testing with perfect cubic structure first...
Cubic structure space group: 221
Cubic structure std under correct SG: 0.00000000 Å

Testing with pyxtal generated structure...
Created structure with space group 161 (R3c)
Structure has 30 sites
Detected space group: 161
Standard deviation under detected space group (161): 0.00000000 Å
Standard deviation under wrong space group (221): 1.95174286 Å
✓ Test PASSED: Both structures show good symmetry properties

SYMMETRY ANALYSIS SUMMARY
Unit test result: PASSED

The symmetry analysis procedure:
1. ✓ Loads structures from JSON
2. ✓ Identifies rhombohedral space groups
3. ✓ Applies symmetry operations to each site
4. ✓ Computes standard deviation considering periodic boundary conditions
5. ✓ Provides quantitative measure of structural symmetry in Angstroms
